In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import altair as alt
import dataframe_image as dfi

In [2]:
#匯入交通部觀光局之公開資料
#資料中的欄位名稱、編碼，皆參考《107年來臺旅客消費及動向調查 (公共版)》之過錄編碼簿

inbound2019 = pd.read_csv('108年來臺旅客消費及動向調查_原始資料.csv')
inbound2019 = pd.DataFrame(inbound2019)

#內含資料屬性混雜的欄位，因不屬於本次分析範圍，暫不處理

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (57,59,61,62,63,64,65) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#針對15個景點紀錄欄位，篩選出曾到訪花蓮景點的旅客
#已知花蓮景點編碼共23項

huavisit= pd.DataFrame()
spotno = [877,878,879] #花蓮'夜市景點'編號877-879
spotcol = []

for j in range(350,370):
    spotno.append(j) #花蓮'其他景點'編號350-369
    
for i in range(1,16):
    spotcol.append(f'sp{i:02d}')  #景點紀錄欄位sp01-sp15
    inbound2019[f'sp{i:02d}'] = pd.to_numeric(inbound2019[f'sp{i:02d}'],
                                              errors='coerce') #欄位dtype從文字轉數字
    huavisit[f'{i}'] = inbound2019[f'sp{i:02d}'].isin(spotno)
    #逐一比對含花蓮景點編號的欄位


In [4]:
#彙整所有含花蓮景點的欄位
#得到在sp01-sp15中，至少有一欄位為花蓮景點
huavisit2019 = inbound2019[huavisit['1']|huavisit['2']|huavisit['3']|huavisit['4']|
            huavisit['5']|huavisit['6']|huavisit['7']|huavisit['8']|
            huavisit['9']|huavisit['10']|huavisit['11']|huavisit['12']|
            huavisit['13']|huavisit['14']|huavisit['15']]

#因陸客未來可能極少訪臺，分析前先剔除居住地'編號2'的旅客資料
huavisit2019 = huavisit2019[huavisit2019['resid']!= 2]

#整理index
huavisit2019 = huavisit2019.reset_index()
huavisit2019.head()

,index,id,mon,stay,freq,plan,purp1,purp2,tt,ad,...,city,age,educ,income,occup,gender,religion,area,nation_o,resid_o
0,28,1080029,1,31,1,180,4,1,4,1,...,.,7,2,8,12,2,4,1,60,60
1,34,1080035,1,6,2,45,1,99,2,1,...,.,4,2,4,2,1,4,1,3,3
2,57,1080058,1,7,6,60,4,1,6,2,...,.,3,1,8,5,2,2,1,55,55
3,91,1080092,1,6,1,90,1,99,1,2,...,.,6,1,8,12,1,2,1,51,51
4,92,1080093,1,12,1,30,1,99,1,1,...,.,6,1,8,12,1,1,1,2,60


In [5]:
#預設花蓮旅遊的市場區隔為6群
clf = KMeans(n_clusters=6) 

#設定Kmeans機器學習演算法和結果輸出欄位
def Kmeans(df, cla):

    clf.fit(df[cla].values.tolist())
    df['分群'] = clf.labels_    
    return df['分群']    

In [6]:
#將花蓮遊客資料，整理為Kmeans用

#資料A :旅客訪臺參加活動19種，資料本身已是獨熱編碼
data_a = huavisit2019.loc[:,'act01':'act19']

#資料B :旅客停留夜數、居住地、來台目的、旅遊型態、到訪花蓮景點
data_b = huavisit2019[['stay','resid','purp1','type']]
spot_col = huavisit2019.loc[:,'sp01':'sp15']
data_b = pd.concat([data_b,spot_col], axis=1)


In [7]:
#將資料B轉換獨熱編碼

#將'停留夜數'分成8階的旅遊天數，轉成獨熱編碼
#1夜<停留夜數<=3夜，即為4天內，以此類推
data_b[f'遊4天以內'] = data_b['stay'].apply(lambda x: x/x if 1<x<=3 else 0)
data_b[f'遊10天以內'] = data_b['stay'].apply(lambda x: x/x if 3<x<=9 else 0)
data_b[f'遊半月以內'] = data_b['stay'].apply(lambda x: x/x if 9<x<=14 else 0)
data_b[f'遊半月至一月內'] = data_b['stay'].apply(lambda x: x/x if 14<x<=29 else 0)
data_b[f'遊1.5月以內'] = data_b['stay'].apply(lambda x: x/x if 29<x<=44 else 0)
data_b[f'遊2月以內'] = data_b['stay'].apply(lambda x: x/x if 44<x<=59 else 0)
data_b[f'遊3月以內'] = data_b['stay'].apply(lambda x: x/x if 59<x<=89 else 0)
data_b[f'遊4月以內'] = data_b['stay'].apply(lambda x: x/x if 89<x<=119 else 0)


#將居住地、來台目的、旅遊型態資料，轉成獨熱編碼
b_col = ['resid','purp1','type']

for col in b_col:
    b_value = data_b[col].value_counts().index #找出單一欄位中不重複的值
    for i in b_value :
        data_b[col+f'_{i}'] = data_b[col].apply(lambda x: x/x if x==i else 0)
        #建立一新欄位，以記錄該值存在的index (存在填1，不存在填0)

#比對居住地、來台目的、旅遊型態之內容值編碼 ,更改為中文欄位名
data_b.rename(columns={'resid_3':'韓國','resid_4':'香港澳門', 'resid_71':'英德法',
                            'resid_60':'美國', 'resid_51':'馬來西亞', 'resid_50':'新加坡',
                            'resid_72':'歐洲其他地區','resid_90':'其他地區',
                            'resid_1':'日本', 'resid_80':'紐澳', 'resid_55':'越南',
                            'resid_52':'印尼', 'resid_54':'泰國', 'resid_59':'印度',
                            'resid_53':'菲律賓'},inplace=True)

    
data_b.rename(columns={'purp1_1':'觀光', 'purp1_4':'探親或訪友', 'purp1_2':'業務',
                            'purp1_7':'其他目的','purp1_6':'醫療', 'purp1_3':'國際會議或展覽',
                            'purp1_5':'求學' },inplace=True)


data_b.rename(columns={'type_1':'行程旅行社包辦', 'type_2':'行程自訂,旅行社包辦',
                            'type_3':'行程自訂,旅行社代訂','type_4':'自由行,含本地旅行社行程',
                            'type_5':'自由行' },inplace=True)


#將15個景點紀錄欄位，轉成獨熱編碼
#先根據index合併15個欄位，依次組成1個新欄位
splist = pd.DataFrame(columns={'allsp',}) 
count = 0

for i in zip(data_b['sp01'].tolist(),data_b['sp02'],data_b['sp03'],
             data_b['sp04'],data_b['sp05'],data_b['sp06']
            ,data_b['sp07'],data_b['sp08'],data_b['sp09']
            ,data_b['sp10'],data_b['sp11'],data_b['sp12']
            ,data_b['sp13'],data_b['sp14'],data_b['sp15']):     
    splist.loc[count,'allsp']=i
    count += 1

#再將新欄位中，一筆資料所含的多個景點編碼，轉成獨熱編碼
for i in spotno :    
        data_b[f'spot{i}'] = splist['allsp'].apply(lambda x: x[0]/x[0] if i in x else 0)
         
              
#比對花蓮景點編碼 ,更改為中文欄位名
data_b.rename(columns={'spot350':'太魯閣,天祥', 'spot351':'遠雄海洋公園',
                        'spot352':'七星潭','spot353':'阿美文化村',
                        'spot354':'鯉魚潭風景特定區','spot355':'坪林兆豐休閒農場',
                        'spot356':'瑞穗紅葉溫泉','spot357':'秀姑巒溪泛舟觀光區',
                        'spot358':'秀山溫泉','spot359':'慕谷慕魚',
                        'spot360':'東華大學','spot361':'花蓮海濱公園',
                        'spot362':'花蓮海岸賞鯨','spot363':'花蓮觀光糖廠',
                        'spot364':'石梯坪','spot365':'花蓮慈濟',
                        'spot366':'馬太鞍濕地生態園區','spot367':'六十石山',
                        'spot368':'花蓮文化創意產業園區','spot369':'台開心農場',
                        'spot877':'南濱夜市','spot878':'東大門夜市',
                        'spot879':'花園夜市',},inplace=True)

In [8]:
#資料A更改欄位名
data_a.rename(columns={'act01':'生態旅遊', 'act02':'運動或賽事', 'act03':'泡溫泉浴',
                        'act04':'拍婚紗或個人藝術照', 'act05':'參觀展覽', 'act06':'參觀藝文表演',
                        'act07':'參觀節慶活動','act08':'購物', 'act09':'逛夜市', 'act10':'參觀古蹟',
                        'act11':'spa.三溫暖', 'act12':'按摩.指壓', 'act13':'護膚.美容.彩繪指甲',
                        'act14':'卡拉ok', 'act15':'夜總會.pub','act16':'遊湖', 'act17':'主題樂園',
                        'act18':'保健醫療', 'act19':'其他活動' },inplace=True)


#合併資料A、B
data = pd.concat([data_b,data_a], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431 entries, 0 to 430
Data columns (total 95 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   stay          431 non-null    int64  
 1   resid         431 non-null    int64  
 2   purp1         431 non-null    int64  
 3   type          431 non-null    int64  
 4   sp01          431 non-null    float64
 5   sp02          431 non-null    float64
 6   sp03          425 non-null    float64
 7   sp04          418 non-null    float64
 8   sp05          393 non-null    float64
 9   sp06          362 non-null    float64
 10  sp07          330 non-null    float64
 11  sp08          283 non-null    float64
 12  sp09          228 non-null    float64
 13  sp10          184 non-null    float64
 14  sp11          134 non-null    float64
 15  sp12          102 non-null    float64
 16  sp13          80 non-null     float64
 17  sp14          57 non-null     float64
 18  sp15          47 non-null     

In [9]:
#用合併後的data製作'花蓮旅客_各類人數統計表A'

hualien_1 = pd.DataFrame(columns=['旅遊天數','統計人數','來台目的',
                                  '各類人數','旅遊型態','人數'])

#各旅遊天數之人數統計
count = 0
for i in data.loc[:,'遊4天以內':'遊4月以內'].columns.tolist():
    hualien_1.loc[count,'旅遊天數'] = i
    hualien_1.loc[count,'統計人數'] = data[f'{i}'][data[f'{i}']== 1].sum()
    count += 1


#來台目的之人數統計
count = 0
for i in data.loc[:,'觀光':'國際會議或展覽'].columns.tolist():
    hualien_1.loc[count,'來台目的'] = i
    hualien_1.loc[count,'各類人數'] = data[f'{i}'][data[f'{i}']== 1].sum()
    count += 1


#旅遊型態之人數統計
count = 0
for i in data.loc[:,'行程自訂,旅行社代訂':'行程自訂,旅行社包辦'].columns.tolist():
    hualien_1.loc[count,'旅遊型態'] = i
    hualien_1.loc[count,'人數'] = data[f'{i}'][data[f'{i}']== 1].sum()
    count += 1

#調整hualien_1的表格設計
hualien_1 = hualien_1.style.hide_index().bar('統計人數', color='#5f9bd6',vmin=0
                                                         ).bar('各類人數', color='#d65f5f',vmin=0
                                                              ).bar('人數', color='#5fd69b',vmin=0
                                                                   ).format({'統計人數': "{0:,.0f}",
                                                                             '各類人數': "{0:,.0f}", 
                                                                             '人數': "{0:,.0f}"})
#匯出'花蓮旅客_各類人數統計表A'圖片
dfi.export(hualien_1,"hualien_A.png")

hualien_1

旅遊天數,統計人數,來台目的,各類人數,旅遊型態,人數
遊4天以內,95,觀光,364,"行程自訂,旅行社代訂",183
遊10天以內,220,探親或訪友,55,行程旅行社包辦,115
遊半月以內,67,其他目的,4,自由行,114
遊半月至一月內,31,業務,4,"自由行,含本地旅行社行程",11
遊1.5月以內,11,求學,2,"行程自訂,旅行社包辦",8
遊2月以內,3,國際會議或展覽,2,nan,nan
遊3月以內,3,nan,nan,nan,nan
遊4月以內,1,nan,nan,nan,nan


In [10]:
#用data製作'花蓮旅客_各類人數統計表B'

hualien_2 = pd.DataFrame(columns=['旅客居住地','統計人數','到訪景點',
                                  '到訪人數','參加活動','參加人數'])

#居住地之人數統計
count = 0
for i in data.loc[:,'韓國':'菲律賓'].columns.tolist():
    hualien_2.loc[count,'旅客居住地'] = i
    hualien_2.loc[count,'統計人數'] = data[f'{i}'][data[f'{i}']== 1].sum()
    count += 1


#景點之人數統計
count = 0
for i in data.loc[:,'南濱夜市':'台開心農場'].columns.tolist():
    hualien_2.loc[count,'到訪景點'] = i
    hualien_2.loc[count,'到訪人數'] = data[f'{i}'][data[f'{i}']== 1].sum()
    count += 1

    
#參加活動之人數統計
count = 0
for i in data.loc[:,'生態旅遊':'其他活動'].columns.tolist():
    hualien_2.loc[count,'參加活動'] = i
    hualien_2.loc[count,'參加人數'] = data[f'{i}'][data[f'{i}']== 1].sum()
    count += 1


#調整hualien_2的表格設計
hualien_2 = hualien_2.style.hide_index().bar('統計人數', color='#5f9bd6',vmin=0
                                                         ).bar('到訪人數', color='#d65f5f',vmin=0
                                                              ).bar('參加人數', color='#5fd69b',vmin=0
                                                                   ).format({'統計人數': "{0:,.0f}",
                                                                             '到訪人數': "{0:,.0f}", 
                                                                             '參加人數': "{0:,.0f}"})
#匯出'花蓮旅客_各類人數統計表B'圖片
dfi.export(hualien_2,"hualien_B.png")

hualien_2

旅客居住地,統計人數,到訪景點,到訪人數,參加活動,參加人數
韓國,99,南濱夜市,0,生態旅遊,108
香港澳門,51,東大門夜市,46,運動或賽事,13
英德法,46,花園夜市,0,泡溫泉浴,7
美國,32,"太魯閣,天祥",405,拍婚紗或個人藝術照,82
馬來西亞,31,遠雄海洋公園,3,參觀展覽,88
新加坡,29,七星潭,116,參觀藝文表演,7
歐洲其他地區,25,阿美文化村,18,參觀節慶活動,5
其他地區,23,鯉魚潭風景特定區,8,購物,398
日本,21,坪林兆豐休閒農場,0,逛夜市,384
紐澳,20,瑞穗紅葉溫泉,14,參觀古蹟,227


In [11]:
#data套用Kmeans進行分群

#先選出獨熱編碼的欄位們
col = data.loc[:,'遊4天以內':'其他活動'].columns.tolist()

#開始分群
data['分群'] = Kmeans(df = data, cla = [x for x in col])

#檢視各群的人數
data['分群'].value_counts()

5    90
2    86
0    83
3    66
1    54
4    52
Name: 分群, dtype: int64

In [12]:
#各群做逐一的性質分析

for c in data['分群'].value_counts().index:
    df = data[data['分群']==c] #依分群的編碼，選取一個群的資料
    seg = []
    count = []
    
    for i in df.columns[19:95].tolist(): #選取群資料中，所有獨熱編碼的欄位名
        for j in df[i].values.tolist(): #選取一欄位中，每一列的值      
            if j == 1: #若該列的值為1
                if not(i in seg): #且欄位名未記錄在seg清單
                    seg.append(i) #新增欄位名
                    count.append(1) #該欄位的計數加一
                else:
                    count[seg.index(i)] += 1 #若已在seg清單，就只是計數加一
                    
    dic = {'群人數':len(df),'性質':seg,'人數':count}
    segdata = pd.DataFrame(dic)
    people = segdata['群人數'][0]
   
    #將一個群的性質分析結果畫成長條圖
    #y軸是該群的性質，x軸是(該性質人數/群人數)%
    #原本Kmeans的分群編碼為0-5，為後續分析便利，名稱改成第1-6群
    
    bars =alt.Chart(segdata, title= f'第{c+1}群_性質分析  人數{people}人'
                   ).transform_joinaggregate(Totalpeople='sum(群人數)',).transform_calculate(
        PercentOfTotal='datum.人數 / datum.群人數').mark_bar().encode(
        alt.X('PercentOfTotal:Q', axis=alt.Axis(format='.0%')),alt.Y(
            '性質:N', sort='-x') ,color=alt.Color("PercentOfTotal:Q", legend=None))
    
     
    text = bars.mark_text(align='left',baseline='middle',dx=3).encode(
        alt.Text('PercentOfTotal:Q', format='.0%'))

     
    globals()[f'chart_{c+1}cluster'] = (bars + text).properties(width=300).configure_axis(labelFontSize=14,titleFontSize=16 )
   

In [13]:
#檢視第1群的性質分析長條圖
chart_1cluster


alt.LayerChart(...)

In [14]:
#檢視第2群的性質分析長條圖
chart_2cluster


alt.LayerChart(...)

In [15]:
#檢視第3群的性質分析長條圖
chart_3cluster

alt.LayerChart(...)

In [16]:
#檢視第4群的性質分析長條圖
chart_4cluster


alt.LayerChart(...)

In [17]:
#檢視第5群的性質分析長條圖
chart_5cluster


alt.LayerChart(...)

In [18]:
#檢視第6群的性質分析長條圖
chart_6cluster


alt.LayerChart(...)

In [20]:
#建立花蓮旅遊市場區隔表

analys_df = pd.DataFrame(columns = ['群','群特性','人數','總人數%','旅遊消費',
                                    '總消費%','最愛花蓮%','消費人數比'])

#使用原先的'huavisit2019'資料集，以計算各群的人數、消費等等
huavisit2019['分群'] = data['分群']

#各群編碼及人數
analys_df['群'] = huavisit2019['分群'].value_counts().index+1
analys_df['人數'] = huavisit2019['分群'].value_counts().values


#計算各群人數佔花蓮遊客的百分比
for i in analys_df['群'].values:
    loc =  analys_df[analys_df['群'] == i]['群'].index
    analys_df['總人數%'][loc] = analys_df['人數'][loc]/analys_df['人數'].sum()*100

<ipython-input-20-9587e8decaab>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analys_df['總人數%'][loc] = analys_df['人數'][loc]/analys_df['人數'].sum()*100
C:\Users\r\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-20-9587e8decaab>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analys_df['總人數%'][loc] = ana

In [21]:
#統計各群的在台的旅遊總消費
#因消費金額以外幣為單位，須針對每一位旅客的消費做匯率換算

#列出旅客的使用幣別
d = huavisit2019['dollar'].unique()
print(d)

[16  5  6  8  3 14  1  2 11  9  4  7 17 19 10]


In [22]:
#將d的15種外幣，對照外幣編碼表、2019年外幣匯率，製作匯率對照資料表
#2019年外幣對臺幣的匯率，參考財政部台北國稅局公布之《外僑綜合所得稅匯率明細表(101年─108年)》

exchange = pd.DataFrame({1:0.2810,#日圓
            2:4.4678,#人民幣
            3:3.9087,#港幣
            4:22.5533,#新加坡元
            5:0.0265,#韓元
            6:7.4540,#馬來西亞幣
            7:0.9829,#泰銖
            8:30.8125,#美元
            9:23.1816,#加拿大幣
            10:39.2225,#英鎊
            11:34.3266,#歐元
            14:21.3741,#澳大利亞幣
            16:1.,#新台幣
            17:0.0022,#印尼盾
            19:0.0013319}#越南盾(2019年匯率平均值)
            ,index= {0,})

In [23]:
#折合台幣，計算各群的旅遊消費
for i in analys_df['群'].values:
    loc = analys_df[analys_df['群'] == i]['群'].index
    
    #各群的消費金額
    fee = huavisit2019[huavisit2019['分群'] == (i-1)]['money'] 
    
    #消費的外幣種類與匯率
    rate = exchange.loc[:,(huavisit2019[huavisit2019['分群'] == (i-1)]['dollar'])]
    
    #金額*匯率=臺幣金額
    cluster = [a* b for a, b in zip(fee, rate.loc[0].values)]
    analys_df['旅遊消費'][loc] = sum(cluster)
    
    
#計算各群的總消費百分比
for i in analys_df['群'].values:
    base = analys_df[analys_df['群'] == i]  
    analys_df['總消費%'][base['群'].index] = base['旅遊消費'].sum()/analys_df['旅遊消費'].sum()*100


<ipython-input-23-dea74f3c589e>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analys_df['旅遊消費'][loc] = sum(cluster)
C:\Users\r\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-23-dea74f3c589e>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analys_df['旅遊消費'][loc] = sum(cluster)
C:\Users\r\AppData\Roami

In [24]:
#計算消費人數比:各群中一個人可以貢獻多少消費比

analys_df['消費人數比']=[a/b for a,b in zip(analys_df['總消費%'],analys_df['總人數%'])]

In [25]:
#用'遊客最愛景點'欄位，計算各群最愛花蓮景點的比例

huavisit2019['spfav'] = pd.to_numeric(huavisit2019['spfav'],errors='coerce')

for i in analys_df['群'].values:
    loc = analys_df[analys_df['群'] == i]['群'].index
    fav = huavisit2019[huavisit2019['分群'] == (i-1)]   
    #最愛花蓮%='最愛景點在花蓮23個景點當中'資料筆數/各群人數
    analys_df['最愛花蓮%'][loc] = len(fav[fav['spfav'].isin (spotno)])/analys_df['人數'][loc]*100


<ipython-input-25-e3e959140a4e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analys_df['最愛花蓮%'][loc] = len(fav[fav['spfav'].isin (spotno)])/analys_df['人數'][loc]*100
C:\Users\r\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-25-e3e959140a4e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analys_df['最愛花

In [29]:
#所有資料依群編碼排序
analys_df = analys_df.sort_values(by='群')

#手動補充分析者對各群的特性描述
analys_df['群特性'] = ['韓國客4天古蹟觀光團','美、港澳客10天探親訪友','歐洲、港澳客10天自由行',
                    '歐洲、紐澳客半個月生態旅行','亞洲客10天觀光團','東南亞客10天花蓮趴趴走']


analys_df

,群,群特性,人數,總人數%,旅遊消費,總消費%,最愛花蓮%,消費人數比
2,1,韓國客4天古蹟觀光團,83,19.2575,1.79232e+06,13.7775,57.8313,0.715436
4,2,美、港澳客10天探親訪友,54,12.529,2.00111e+06,15.3825,57.4074,1.227748
1,3,歐洲、港澳客10天自由行,86,19.9536,3.11397e+06,23.937,62.7907,1.199634
3,4,歐洲、紐澳客半個月生態旅行,66,15.3132,2.24801e+06,17.2804,45.4545,1.128465
5,5,亞洲客10天觀光團,52,12.065,1.13088e+06,8.69306,30.7692,0.720521
0,6,東南亞客10天花蓮趴趴走,90,20.8817,2.72272e+06,20.9295,54.4444,1.002290


In [30]:
#繪製各群的最愛花蓮%、消費人數比之定位圖

bar = alt.Chart(analys_df).mark_circle().encode(
    alt.X('average(最愛花蓮%):Q',title = '最愛花蓮%',scale=alt.Scale(zero=False)),
    alt.Y('average(消費人數比):Q',title = '消費人數比',scale=alt.Scale(zero=False, padding=1)),
    alt.Size('人數:Q'), color='群:N')

#紅線表示y軸平均值
rule = alt.Chart(analys_df).mark_rule(color='red').encode(
    alt.Y('mean(消費人數比):Q' ,title = '')
)
#紅線表示x軸平均值
rule2 = alt.Chart(analys_df).mark_rule(color='red').encode(
    alt.X('mean(最愛花蓮%):Q',title = '' ))

( rule+rule2+bar).properties(width=450, height=250,title='花蓮旅遊市場定位分析   最愛花蓮v.s.消費人數比'). configure_axis(grid=False)


alt.LayerChart(...)

In [31]:
#調整花蓮旅遊市場區隔表之設計
analys_df = analys_df.style.hide_index().format({'總人數%': "{0:,.2f}",
                                                 '旅遊消費': "{0:,.0f}",
                                                 '總消費%': "{0:,.2f}",
                                                 '最愛花蓮%': "{0:,.2f}",
                                                 '消費人數比': "{0:,.1f}"})

#匯出'花蓮旅遊市場區隔表'圖片
dfi.export(analys_df,"analys_df.png")

analys_df

群,群特性,人數,總人數%,旅遊消費,總消費%,最愛花蓮%,消費人數比
1,韓國客4天古蹟觀光團,83,19.26,"1,792,322",13.78,57.83,0.7
2,美、港澳客10天探親訪友,54,12.53,"2,001,105",15.38,57.41,1.2
3,歐洲、港澳客10天自由行,86,19.95,"3,113,968",23.94,62.79,1.2
4,歐洲、紐澳客半個月生態旅行,66,15.31,"2,248,014",17.28,45.45,1.1
5,亞洲客10天觀光團,52,12.06,"1,130,881",8.69,30.77,0.7
6,東南亞客10天花蓮趴趴走,90,20.88,"2,722,718",20.93,54.44,1.0


In [33]:
#製作'第3群旅客全臺最愛景點前十名'表單

spotlist = []
spotvisit = []
cluster5 = huavisit2019[huavisit2019['分群'] == (3-1)]   

for i in spotcol:
    spot = cluster5[i].value_counts().index.tolist()
    visit = cluster5[i].value_counts().values.tolist()
    for j in spot:
        spotlist.append(j)
    for v in visit:
        spotvisit.append(v)

spot10 = pd.DataFrame({'景點':spotlist,'到訪人數':spotvisit})
spot10 = spot10.groupby('景點').sum().sort_values(by='到訪人數', ascending=False).head(10).reset_index()

#比對景點之內容值編碼 ,新增中文景點名
spot10['景點名']=['太魯閣,天祥','台北101','士林夜市','九份','中正紀念堂','墾丁國家公園','艋舺龍山寺',
              '故宮博物院','西門町','象山',]

spot10

,景點,到訪人數,景點名
0,350.0,80,"太魯閣,天祥"
1,1.0,53,台北101
2,801.0,48,士林夜市
3,65.0,44,九份
4,3.0,30,中正紀念堂
5,315.0,28,墾丁國家公園
6,8.0,27,艋舺龍山寺
7,2.0,27,故宮博物院
8,5.0,26,西門町
9,56.0,26,象山


In [35]:
#製作'第3群旅客全臺最愛景點前十名'圖表

alt.Chart(spot10, title='歐洲、港澳客10天自由行  全臺最愛景點前十名').mark_point(size=200,).encode(
    alt.X(
        '到訪人數:Q',
        title='景點到訪人數',
        axis=alt.Axis(grid=False)
    ),
    alt.Y(
        '景點名:N',
        title="",
        sort='-x',
        axis=alt.Axis(grid=True)
    ),
    alt.Color("景點名:N", legend=None)).configure_view(stroke="transparent"
    ).properties(height=alt.Step(35)).configure_axis(labelFontSize=16,titleFontSize=16)

alt.Chart(...)